## Proyecto final
1. Recuerde que se divide en dos partes:
    - Informe escrito y código
    - Presentación
2. Habrá entregas parciales, las cuales son un prerequisito para el proyecto final. 
    - La calificación de dichas entregas entran en el apartado de tares y trabajo en clase.
    - Primera entrega: 
        + Introducción
        + Antecedentes
        + Objetivos
        + Planteamiento del problema
        + EDA
        + Data Wrangling
        + Dataset final a trabajar
3. Fecha de entrega: 26 de Octubre de 2022 a las 19:55

## Repaso de los conceptos vistos la clase anterior

# Manifiestos en Kubernetes
Un archivo de manifiesto de Kubernetes incluye instrucciones en un archivo yaml o json que especifican cómo desplegar una aplicación al nodo o nodos en un cluster de Kubernetes. Las instrucciones incluyen información sobre el despliegue de Kubernetes, el servicio Kubernetes y otros objetos de Kubernetes que se crearán en el cluster. También se suele hacer referencia al manifiesto como especificación de pod o como archivo deployment.yaml (aunque se permiten otros nombres de archivo).

![](https://i.stack.imgur.com/kyLFz.png)

- Iniciar `docker` y `minikube`
```
minikube start
```

## POD's
- Crear un pod sencillo

```
apiVersion: v1
kind: Pod
metadata:
  name: hello-pod
spec:
  containers:
  - name: hello
    image: gcr.io/google-samples/hello-app:1.0
```

- Ejecutar ese deployment
```
kubectl apply -f .yaml
```


- Ahora vamos a crear otro pod un poco más completo
```
apiVersion: v1
kind: Pod
metadata:
  name: deployment
spec:
  containers:
  - name: hello
    image: nginx:alpine
    env:
    - name: MI_VARIABLE
      value: "variable1"
    - name: MI_OTRA_VARIABLE
      value: "esta es una contrasena"
    - name: DD_AGENT_HOST
      valueFrom:
        fieldRef:
          fieldPath: status.hostIP
    resources:
      requests:
        memory: "64Mi"
        cpu: "200m"
      limits:
        memory: "128Mi"
        cpu: "500m"
    readinessProbe:
      httpGet:
        path: /
        port: 80
      initialDelaySeconds: 5
      periodSeconds: 10
    livenessProbe:
      tcpSocket:
        port: 80
      initialDelaySeconds: 15
      periodSeconds: 20
    ports:
    - containerPort: 80
```

- Ahora vamos a borrarlo... Se debería volver a "levantar" solo?
```
kubectl delete pod deployment.yaml
```

## Deployments
 Es un template para crear PODS

- Crear un deployment:
```
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-deployment
spec:
  selector:
    matchLabels:
      app: nginx
  replicas: 2
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: nginx:alpine
        env:
        - name: MI_VARIABLE
          value: "variable1"
        - name: MI_OTRA_VARIABLE
          value: "esta es una contrasena"
        - name: DD_AGENT_HOST
          valueFrom:
            fieldRef:
              fieldPath: status.hostIP
        resources:
          requests:
            memory: "64Mi"
            cpu: "200m"
          limits:
            memory: "128Mi"
            cpu: "500m"
        readinessProbe:
          httpGet:
            path: /
            port: 80
          initialDelaySeconds: 5
          periodSeconds: 10
        livenessProbe:
          tcpSocket:
            port: 80
          initialDelaySeconds: 15
          periodSeconds: 20
        ports:
        - containerPort: 80
```
- Ejecutar el deployment
```
kubectl apply -f deployment.yaml
```

- Ahora sí... Qué pasa si borramos un pod?
```
kubectl delete pod deployment.yaml
```

## Load Balancer

- Vamos a crear un deployment con balanceador de carga
```
apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello
spec:
  replicas: 3
  selector:
    matchLabels:
      role: hello
  template:
    metadata:
      labels:
        role: hello
    spec:
      containers:
      - name: hello
        image: gcr.io/google-samples/hello-app:1.0
        ports:
        - containerPort: 8080

---
apiVersion: v1
kind: Service
metadata:
  name: hello
spec:
  ports:
  - port: 8080
    targetPort: 8080
  selector:
    role: hello
```

- Revisar las ips
```
kubectl describe svc hello
kubectl get pods -o wide
```

- Eliminar de nuevo un pod

- Vamos a crear un pod de ubuntu para conectarnos
```
apiVersion: v1
kind: Pod
metadata:
  name: ubuntu
spec:
  containers:
  - name: ubuntu
    image: ubuntu
    args:
    - sleep
    - infinity
```
- Ingresamos al pod ubuntu
```
kubectl exec -it ubuntu -- bash
apt-get update
apt-get upgrade
apt install curl
curl http://hello:8080
```
